In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

%cd "/content/drive/MyDrive/GSDS/2022-1/MLDL1/DL/homework3"

Mounted at /content/drive
/content/drive/MyDrive/GSDS/2022-1/MLDL1/DL/homework3


In [2]:
import numpy as np
import pickle
with open('testcases.pkl', 'rb') as f:
  tc_list = pickle.load(f)

i = 0
for image, filter, stride, padding, output in tc_list:
    print(f"===================== Image {i} =====================")
    print(f"Image:\n{image}")
    print(f"Filter:\n{filter}")
    print(f"Stride: {stride}")
    print(f"Padding: {padding}")
    print(f"Output:\n{output}\n")
    i += 1

===================== Image 0 =====================
Image:
[[ 1  2  3  2  1]
 [ 2  3  4  5  6]
 [-1 -2 -3 -4 -5]
 [ 0  0  1  0  0]
 [ 7  1  7  1  7]]
Filter:
[[1 0]
 [0 1]]
Stride: 3
Padding: 0
Output:
[[4. 8.]
 [1. 7.]]

===================== Image 1 =====================
Image:
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
Filter:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Stride: 2
Padding: 1
Output:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

===================== Image 2 =====================
Image:
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
Filter:
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
Stride: 3
Padding: 2
Output:
[[1. 3. 1.]
 [3. 9. 3.]
 [1. 3. 1.]]

===================== Image 3 =====================
Image:
[[ 1  2  3  2  1]
 [ 2  3  4  5  6]
 [-1 -2 -3 -4 -5]
 [ 0  0  1  0  0]
 [ 7  1  7  1  7]]
Filter:
[[1 0]
 [0 1]]
Stride: 1
Padding: 3
Output:
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.

In [3]:
def Convolution(image, filter, stride=1, padding=0):
    ## To avoid division by zero
    if stride <= 0:
        return None
    ## compute the shape of final output
    cond0 = (image.shape[0] + (2*padding) - filter.shape[0]) / stride
    cond1 = (image.shape[1] + (2*padding) - filter.shape[1]) / stride 

    shape0 = int(cond0) + 1
    shape1 = int(cond1) + 1

    ## error-checking 2
    # when filter is larger than the given image
    if (cond0 < 0) or (cond1 < 0):
        return None
    # when stride is not compatible
    if (cond0 - int(cond0) != 0) or (cond1 - int(cond1) != 0):
        return None

    ## transform the original image into padded one
    transform = np.zeros(shape=(image.shape[0]+(2*padding), image.shape[1]+(2*padding)))
    transform[padding:(transform.shape[0]-padding), padding:(transform.shape[1]-padding)] = image

    ## convolution
    output = np.zeros(shape=(shape0, shape1))
    idx = 0
    for i in range(shape0):
        jdx = 0
        for j in range(shape1):
            piece = transform[idx:(idx+filter.shape[0]), jdx:(jdx+filter.shape[1])]
            convsum = np.sum(piece * filter, dtype=np.float64)
            output[i][j] = convsum
            jdx += stride
        idx += stride
    
    return output

In [4]:
score = 0
for idx, (image, pattern, stride, padding, output_sol) in enumerate(tc_list):
  print("==========test case #"+str(idx+1)+"==========")
  try:
      output = Convolution(image, pattern, stride, padding)
  except ZeroDivisionError:
      print("  Exception: Zero Division Error")
      continue
  except ValueError:
      print("  Exception: Value Error")
      continue
  except IndexError:
      print("  Exception: Index Error")
      continue

  if output_sol is None and output is None:
    print('  Correct (case 1)'.format(stride, padding))
    score += 1
  elif output is None:
    print('  Wrong (case 2: output is None)'.format(stride, padding))
  elif output_sol is None:
    print('  Wrong (case 3: output should be None but not. {})'.format(stride, padding, output))
  elif np.array(output).all() == output_sol.all():
    print('  Correct (case 4)'.format(stride, padding))
    score += 1
  else:
    print('  Wrong (case 5: received != expected. {} {})'.format(stride, padding, output, output_sol))
print("Final Score:", score)

==========test case #1==========
  Correct (case 4)
==========test case #2==========
  Correct (case 4)
==========test case #3==========
  Correct (case 4)
==========test case #4==========
  Correct (case 4)
==========test case #5==========
  Correct (case 1)
==========test case #6==========
  Correct (case 1)
==========test case #7==========
  Correct (case 4)
==========test case #8==========
  Correct (case 1)
==========test case #9==========
  Correct (case 1)
==========test case #10==========
  Correct (case 4)
==========test case #11==========
  Correct (case 1)
==========test case #12==========
  Correct (case 1)
==========test case #13==========
  Correct (case 4)
==========test case #14==========
  Correct (case 4)
==========test case #15==========
  Correct (case 1)
==========test case #16==========
  Correct (case 4)
==========test case #17==========
  Correct (case 4)
==========test case #18==========
  Correct (case 1)
==========test case #19==========
  Correct (case 4)
==

In [5]:
import cv2
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets

In [6]:
shape = (32, 32, 3)
x_ = np.random.randn(*shape)
x = torch.Tensor(x_)
print(x)
print(x.shape)

tensor([[[ 4.8642e-01, -3.9221e-01,  1.3460e+00],
         [ 5.7568e-01, -4.9587e-01,  1.0763e-01],
         [-5.3817e-02,  1.2142e+00,  1.0456e-01],
         ...,
         [ 8.0413e-02, -2.8272e-01,  1.5267e+00],
         [-1.4753e+00, -6.7434e-01, -4.9255e-01],
         [ 1.6555e+00,  1.9563e+00,  3.3217e-01]],

        [[ 4.6796e-04, -1.4781e+00,  4.2479e-01],
         [ 1.1476e-01,  2.7347e-01, -1.4424e+00],
         [ 3.9537e-01, -1.6946e-01,  1.3089e+00],
         ...,
         [-4.0842e-01, -2.4241e-01, -7.4538e-01],
         [ 1.8287e+00,  2.4846e-01, -1.6585e+00],
         [ 1.8757e-01,  3.0639e+00,  8.3432e-01]],

        [[-4.1609e-01, -4.9040e-02,  1.6845e+00],
         [-1.3495e+00, -1.1323e+00, -8.9640e-01],
         [ 1.2940e+00, -2.2213e+00,  1.0728e+00],
         ...,
         [ 5.1032e-01,  1.7877e-01,  7.6083e-01],
         [ 1.9971e-01,  7.7456e-01,  9.6050e-03],
         [ 2.7001e-01,  6.0769e-01,  2.0452e+00]],

        ...,

        [[ 4.8368e-01,  2.5286e+00,  7

In [7]:
view = x.view(-1, 32*32*3)
print(view.size())
print(view.dim())

torch.Size([1, 3072])
2


In [8]:
view = x.view(-1, 32*32)
print(view.size())
print(view.dim())

torch.Size([3, 1024])
2


In [9]:
view = x.view(-1, 4*32*3)
print(view.size())
print(view.dim())

torch.Size([8, 384])
2


In [10]:
from IPython.display import display

bnorm = nn.BatchNorm2d(64)
display(bnorm)

BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [11]:
# display(bnorm.item())

AttributeError: ignored